## Aufgabe 1 a - daten lesen

In [29]:
import pandas as pd
import math

# Bestehende Kundendaten
kunden_alt = {
    'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'Monatsumsatz': [80, 120, 60, 150, 200, 220, 300, 320, 280, 400, 420, 380, 500, 480, 520],
    'Bestellungen': [1, 2, 2, 3, 4, 4, 5, 6, 5, 7, 8, 7, 9, 9, 10],
    'Kategorie': ['gering', 'gering', 'gering', 'gering', 'mittel', 'mittel', 'mittel', 'mittel', 'mittel', 'hoch', 'hoch', 'hoch', 'hoch', 'hoch', 'hoch']
}

df_kunden_alt = pd.DataFrame(kunden_alt)
print("Bestehende Kundendaten:")
print(df_kunden_alt)

Bestehende Kundendaten:
    ID  Monatsumsatz  Bestellungen Kategorie
0    1            80             1    gering
1    2           120             2    gering
2    3            60             2    gering
3    4           150             3    gering
4    5           200             4    mittel
5    6           220             4    mittel
6    7           300             5    mittel
7    8           320             6    mittel
8    9           280             5    mittel
9   10           400             7      hoch
10  11           420             8      hoch
11  12           380             7      hoch
12  13           500             9      hoch
13  14           480             9      hoch
14  15           520            10      hoch


In [30]:
# Prospect Kunden aus tabelle 2
kunden_neu = {
    'ID': [16, 17, 18, 19],
    'Monatsumsatz': [90, 170, 350, 460],
    'Bestellungen': [1, 4, 6, 8]
}

df_kunden_neu = pd.DataFrame(kunden_neu)
print("Neue Kundendaten:")
print(df_kunden_neu)

Neue Kundendaten:
   ID  Monatsumsatz  Bestellungen
0  16            90             1
1  17           170             4
2  18           350             6
3  19           460             8


## Distanzmetrik
- erstmal die euklidische

In [31]:
def euklidische_distanz(x1, y1, x2, y2):
    distanz = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    return distanz

## Funktion für die KNN Klassifikation

In [32]:

def knn_algorithmus(neu_umsatz, neu_bestellungen, k):

    distanzen = []
    
    # Distanz zu allen bestehenden Kunden berechnen
    for i in range(len(df_kunden_alt)):
        umsatz = df_kunden_alt.loc[i, 'Monatsumsatz']
        bestellungen = df_kunden_alt.loc[i, 'Bestellungen']
        kategorie = df_kunden_alt.loc[i, 'Kategorie']
        kunde_id = df_kunden_alt.loc[i, 'ID']
        
        distanz = euklidische_distanz(neu_umsatz, neu_bestellungen, umsatz, bestellungen)
        distanzen.append({'ID': kunde_id, 'Distanz': distanz, 'Kategorie': kategorie})
    
    # Distanz sortieren
    distanzen_sortiert = sorted(distanzen, key=lambda x: x['Distanz'])
    
    # Selektieren 
    k_nachbarn = distanzen_sortiert[:k]
    
    return k_nachbarn, distanzen_sortiert

## Klassifikation für k=5 und k=3

In [33]:
# Klassifikation für beide k-Werte
for k in [5, 3]:
    print(f"\n{'='*50}")
    print(f"K = {k}")
    print(f"{'='*50}")
    
    for i in range(len(df_kunden_neu)):
        kunde_id = df_kunden_neu.loc[i, 'ID']
        umsatz = df_kunden_neu.loc[i, 'Monatsumsatz']
        bestellungen = df_kunden_neu.loc[i, 'Bestellungen']
        
        print(f"\n\n >->->->-> Für den Kunden {kunde_id}")
        print(f"\nKunde {kunde_id}: Umsatz={umsatz}€, Bestellungen={bestellungen}")
        
        k_nachbarn, alle_distanzen = knn_algorithmus(umsatz, bestellungen, k)
        
        print(f"\nDie nächsten Nachbarn:")
        for nachbar in k_nachbarn:
            print(f"  Kunde {nachbar['ID']}: Distanz={nachbar['Distanz']:.2f}, Kategorie={nachbar['Kategorie']}")
        
        # Kategorien zählen
        summe_gering = sum(1 for n in k_nachbarn if n['Kategorie'] == 'gering')
        summe_mittel = sum(1 for n in k_nachbarn if n['Kategorie'] == 'mittel')
        summe_hoch = sum(1 for n in k_nachbarn if n['Kategorie'] == 'hoch')
        
        print(f"\nVerteilung: gering={summe_gering}, mittel={summe_mittel}, hoch={summe_hoch}")
        
        # Kategorie mit den meisten Stimmen
        if summe_hoch >= summe_mittel and summe_hoch >= summe_gering:
            kategorie = 'hoch'
        elif summe_mittel >= summe_gering:
            kategorie = 'mittel'
        else:
            kategorie = 'gering'
        
        print(f">>> Klassifikation: {kategorie}")


K = 5


 >->->->-> Für den Kunden 16

Kunde 16: Umsatz=90€, Bestellungen=1

Die nächsten Nachbarn:
  Kunde 1: Distanz=10.00, Kategorie=gering
  Kunde 2: Distanz=30.02, Kategorie=gering
  Kunde 3: Distanz=30.02, Kategorie=gering
  Kunde 4: Distanz=60.03, Kategorie=gering
  Kunde 5: Distanz=110.04, Kategorie=mittel

Verteilung: gering=4, mittel=1, hoch=0
>>> Klassifikation: gering


 >->->->-> Für den Kunden 17

Kunde 17: Umsatz=170€, Bestellungen=4

Die nächsten Nachbarn:
  Kunde 4: Distanz=20.02, Kategorie=gering
  Kunde 5: Distanz=30.00, Kategorie=mittel
  Kunde 6: Distanz=50.00, Kategorie=mittel
  Kunde 2: Distanz=50.04, Kategorie=gering
  Kunde 1: Distanz=90.05, Kategorie=gering

Verteilung: gering=3, mittel=2, hoch=0
>>> Klassifikation: gering


 >->->->-> Für den Kunden 18

Kunde 18: Umsatz=350€, Bestellungen=6

Die nächsten Nachbarn:
  Kunde 8: Distanz=30.00, Kategorie=mittel
  Kunde 12: Distanz=30.02, Kategorie=hoch
  Kunde 7: Distanz=50.01, Kategorie=mittel
  Kunde 10: Distanz

## Aufgabe b

- Man könnte versuchen die beiden numerischen merkmale zu normalisieren. 
- bspw. per minmax für die anzahl der bestellungen
- und. das selbe für den monatsumsatz (oder auch eventuell logarithmisieren)

## Aufgabe c

Ich würde mich für die SVC matrix entscheiden
Da dass genau den sonderanforderungen erfüllt. 
- Es gibt genau keine hohe wichtigkeitskunden, die als mittel oder gering klassifziert wurden. Und auch keine mittleren, die als gering klassifziert wurden
- es überschätzt die kunden, dass heißt, dass wenn dan nur die wichtigkeit größer eingeschätzt wird, als sie in der realität ist
-> sollten keine kunden verägert werden